In [1]:
from transformers import pipeline
import torch

2023-10-03 23:50:50.642940: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-03 23:50:50.642975: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-03 23:50:50.642996: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-03 23:50:50.647629: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-03 23:50:51.405362: W tensorflow/compiler/

In [2]:
classifier = pipeline("sentiment-analysis")
results = classifier("This is cool")
results

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998584985733032}]

In [3]:
# We give a list to the classifier now
results = classifier(["NLP is nice", "It's a lot of work"])
results

[{'label': 'POSITIVE', 'score': 0.9997960925102234},
 {'label': 'POSITIVE', 'score': 0.9995623230934143}]

In [4]:
# use a specific model and tokenizer
model_name = "distilbert-base-uncased-finetuned-sst-2-english" # trained on the sentiment analysis treebank v.2 --> link
# explain how to get models from the hub

classifier = pipeline("sentiment-analysis", model=model_name)

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(model_name) # this time we need to be a bit more specific, because the model requires a tokenizer adapted to the model. 
tokenizer = AutoTokenizer.from_pretrained(model_name) # what is from_pretrained?

In [7]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [8]:
tokens = tokenizer.tokenize("Another cool sentence to demonstrate something.")
token_ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = tokenizer("Another cool senteance to demonstrate something.")

In [9]:
print(f' Tokens:{tokens}')
print(f' Token IDs: {token_ids}')
print(f' input_ids:{input_ids}')

 Tokens:['another', 'cool', 'sentence', 'to', 'demonstrate', 'something', '.']
 Token IDs: [2178, 4658, 6251, 2000, 10580, 2242, 1012]
 input_ids:{'input_ids': [101, 2178, 4658, 2741, 11219, 3401, 2000, 10580, 2242, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [10]:
X_train = ["Another cool sentence to demonstrate something.",
           "All I need is two sentences."]
batch = tokenizer(X_train, padding=True, truncation=True, max_length=512, return_tensors="pt") #pt for pyTorch

In [11]:
print(batch)

{'input_ids': tensor([[  101,  2178,  4658,  6251,  2000, 10580,  2242,  1012,   102],
        [  101,  2035,  1045,  2342,  2003,  2048, 11746,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [12]:
with torch.no_grad():
    outputs = model(**batch)
    print(outputs)
    predictions = torch.softmax(outputs.logits, dim=1)
    print(predictions)
    labels = torch.argmax(predictions, dim=1)
    print(labels)
    labels = [model.config.id2label[label_id] for label_id in labels.tolist()]
    print(labels)

SequenceClassifierOutput(loss=None, logits=tensor([[-3.9261,  4.2183],
        [ 2.8756, -2.4102]]), hidden_states=None, attentions=None)
tensor([[2.9026e-04, 9.9971e-01],
        [9.9496e-01, 5.0377e-03]])
tensor([1, 0])
['POSITIVE', 'NEGATIVE']


In [13]:
save_directory = "your_dir"
tokenizer.save_pretrained(save_directory)
model. save_pretrained(save_directory)

tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModelForSequenceClassification.from_pretrained(save_directory)
